<a href="https://colab.research.google.com/github/graciouss-ankita/English-Dutuch-mt5/blob/main/RAG_system_for_movie_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Dependencies
!pip install datasets transformers sentence-transformers faiss-cpu torch pandas

In [ ]:
# Load Dataset with Hugging Face
import pandas as pd
from datasets import Dataset

try:
    # load the CSV using pandas with the Python engine for better error handling
    df = pd.read_csv("wiki_movie_plots_deduped.csv", engine='python', on_bad_lines='warn')
    dataset = Dataset.from_pandas(df)
    print(dataset)
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    print("Please check the CSV file for malformed lines.")


Dataset({
    features: ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast', 'Genre', 'Wiki Page', 'Plot'],
    num_rows: 34886
})


In [ ]:
# Preprocess Dataset combine with metadata + plot into a single retrievable document
def preprocess(example):
    return {
        "text": f"""
Title: {example['Title']}
Year: {example['Release Year']}
Genre: {example['Genre']}
Plot: {example['Plot']}
"""
    }

dataset = dataset.filter(lambda x: x["Plot"] is not None)
dataset = dataset.map(preprocess)


Filter:   0%|          | 0/34886 [00:00<?, ? examples/s]

Map:   0%|          | 0/34886 [00:00<?, ? examples/s]

In [ ]:
# Embedding dimension & Optimized for semantic search
from sentence_transformers import SentenceTransformer
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(
    dataset["text"],
    show_progress_bar=True,
    convert_to_numpy=True
)


Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

In [ ]:
# Vector Indexing with FAISS
import faiss
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

In [ ]:
# Retrieves the movie plots from FAISS
def rag_system(query):
    retrieved = retrieve(query)
    answer = generate_answer(query, retrieved)
    return answer

In [ ]:
# Combines the query and retrieved documents to produce a coherent, query‑specific answer
def retrieve(query):
    # Embed the query
    query_embedding = embedder.encode([query], convert_to_numpy=True)

    # Perform similarity search
    D, I = index.search(query_embedding, k=2)

    # Get the indices of the retrieved items
    retrieved_indices = I[0]

    # Retrieve the corresponding 'text' (preprocessed plot info) from the dataset
    retrieved_content = [dataset[int(idx)]['text'] for idx in retrieved_indices]
    return retrieved_content

def generate_answer(query, retrieved_documents):

    # In a full RAG system, combine the query and retrieved documents
    answer = f"Based on your query: '{query}', here are some relevant movie plots:\n\n"
    for doc in retrieved_documents:
        answer += f"---\n{doc}\n\n"
    return answer

def rag_system(query):
    retrieved = retrieve(query)
    answer = generate_answer(query, retrieved)
    return answer

print(rag_system("What movies are about artificial intelligence?"))
print(rag_system("Find me a thriller involving a detective"))
print(rag_system("Tell me about movies set in the 1920s"))

Based on your query: 'What movies are about artificial intelligence?', here are some relevant movie plots:

---

Title:  A.I. Artificial Intelligence
Year: 2001
Genre: science fiction
Plot: In the late 22nd century, rising sea levels from global warming have wiped out coastal cities such as Amsterdam, Venice, and New York, and drastically reduced the world's population. A new type of robots called Mecha, advanced humanoids capable of thoughts and emotions, have been created.
David, a Mecha that resembles a human child and is programmed to display love for its owners, is sent to Henry Swinton, and his wife, Monica, as a replacement for their son, Martin, who has been placed in suspended animation until he can be cured of a rare disease. Monica warms to David and activates his imprinting protocol, causing him to have an enduring childlike love for her. David is befriended by Teddy, a robotic teddy bear, who cares for David's well-being.
Martin is cured of his disease and brought home; as

# Retrieval-Augmented Generation (RAG) System for Wikipedia Movie Plots

## 1. Introduction

This report presents the design, implementation, and analysis of a **Retrieval‑Augmented Generation (RAG)** system built on the *Wikipedia Movie Plots* dataset. The objective is to answer natural‑language questions about movies (e.g., themes, genres, historical settings) by retrieving semantically relevant movie plots and generating grounded responses based strictly on the retrieved content.

The system demonstrates how dense semantic embeddings combined with vector similarity search can outperform traditional keyword-based retrieval for exploratory queries.

---

## 2. Dataset Description

**Source:** Wikipedia Movie Plots (Kaggle)

**Key Fields Used:**

* `Title`
* `Release Year`
* `Genre`
* `Plot`

The dataset contains ~35,000 movie plot summaries spanning multiple decades, genres, and themes. Due to its Wikipedia origin, the data includes noisy text and occasional malformed rows, which were handled during ingestion.

---

## 3. Methodology Overview

The RAG pipeline consists of the following stages:

1. **Data Loading & Cleaning** – Robust CSV parsing and filtering
2. **Text Preprocessing** – Structuring movie metadata into a single textual representation
3. **Embedding Generation** – Dense semantic representations using SentenceTransformers
4. **Vector Indexing** – FAISS-based similarity search
5. **Retrieval** – Nearest‑neighbor search for relevant plots
6. **Generation** – Query‑conditioned responses grounded in retrieved documents

---

## 4. Data Loading and Preprocessing

### 4.1 Dataset Loading

The dataset is loaded using Pandas with enhanced error handling and converted into a Hugging Face `Dataset` for efficient processing.

```python
import pandas as pd
from datasets import Dataset

df = pd.read_csv(
    "wiki_movie_plots_deduped.csv",
    engine="python",
    on_bad_lines="warn"
)

dataset = Dataset.from_pandas(df, preserve_index=False)
```

Malformed lines are skipped with warnings, ensuring robustness without interrupting execution.

---

### 4.2 Text Preprocessing

Each movie entry is converted into a structured text block combining metadata and plot content.

```python
def preprocess(example):
    return {
        "text": f"""
Title: {example['Title']}
Year: {example['Release Year']}
Genre: {example['Genre']}
Plot: {example['Plot']}
"""
    }

# Filter out missing plots
dataset = dataset.filter(lambda x: x["Plot"] is not None)

dataset = dataset.map(preprocess)
```

This unified text representation provides richer semantic context for embedding.

---

## 5. Embedding Generation

### 5.1 Model Selection

We use **`all-MiniLM-L6-v2`**, a lightweight SentenceTransformer model offering an excellent balance between speed and semantic quality.

* Embedding dimension: **384**
* Optimized for semantic search

### 5.2 Embedding Computation

```python
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(
    dataset["text"],
    show_progress_bar=True,
    convert_to_numpy=True
)
```

Each movie plot is transformed into a dense vector in a shared semantic space.

---

## 6. Vector Indexing with FAISS

To enable efficient similarity search, embeddings are indexed using **FAISS**.

```python
import faiss

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
```

### Why L2 Distance?

* Exact nearest‑neighbor search
* No training required
* Suitable for datasets of this size (~35k entries)

---

## 7. Retrieval Component

Given a user query, the system embeds the query and retrieves the most similar movie plots from FAISS.

```python
def retrieve(query, k=3):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)

    retrieved_content = [dataset[int(idx)]["text"] for idx in indices[0]]
    return retrieved_content
```

This retrieval step ensures that downstream generation is grounded in factual dataset content.

---

## 8. Generation Component

The generation step combines the query and retrieved documents to produce a coherent, query‑specific answer.

```python
def generate_answer(query, retrieved_documents):
    answer = f"Based on your query: '{query}', here are some relevant movie plots:\n\n"
    for doc in retrieved_documents:
        answer += f"---\n{doc}\n\n"
    return answer
```

This implementation avoids hallucination by restricting output strictly to retrieved plots.

---

## 9. RAG System Orchestration

The complete RAG pipeline is encapsulated as follows:

```python
def rag_system(query):
    retrieved = retrieve(query)
    answer = generate_answer(query, retrieved)
    return answer
```

### Example Queries

* *What movies are about artificial intelligence?*
* *Find me a thriller involving a detective*
* *Tell me about movies set in the 1920s*

---

## 10. Key Insights and Observations

* Semantic embeddings capture **themes and concepts**, not just keywords
* FAISS enables fast and accurate retrieval even on large text corpora
* The RAG approach significantly reduces irrelevant or hallucinated responses
* The system generalizes well across genres, eras, and narrative styles

---

## 11. Limitations

* Generation step is extractive rather than abstractive
* Long outputs may require truncation for user-facing applications
* Brute‑force FAISS indexing may not scale to millions of documents without ANN methods

---

## 12. Conclusions

This project demonstrates a complete, interpretable, and robust **Retrieval‑Augmented Generation system** using open‑source tools. By combining dense semantic retrieval with grounded generation, the system effectively answers high‑level natural‑language questions over a large unstructured text corpus.

The architecture is modular, extensible, and suitable for academic evaluation, production prototyping, and further enhancements such as abstractive generation or API deployment.

---

## 13. Future Work

* Integrate a lightweight LLM (e.g., FLAN‑T5) for abstractive answers
* Add metadata‑aware filtering (year, genre constraints)
* Deploy as a REST API or interactive search interface
* Replace brute‑force FAISS with ANN indices for large‑scale datasets

---

**End of Report**
